# Model training
Training the actual model that we'll deploy after getting its architecture from the previous step (nn_exploration).

In [1]:
from utils import *

In [2]:
import pandas as pd

# Data importation

dataframe = pd.read_csv('data/data.csv')

#Getting the columns
columns = dataframe.columns.values

#Cleaning the dataframe
df = dataframe.drop(columns = columns[0:2])


In [3]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split, TensorDataset

#Defining input features and target feature
input_cols = columns[2:-1]
targets = columns[-1]

#Getting the tensors
inputs, targets = dataframe_to_torch(df,input_cols, targets)

#Creating the dataset
dataset = TensorDataset(inputs, targets)

#Test:15% training: 85%

test_size = round(0.15*len(dataset))


train_ds, test_ds = random_split(dataset , [len(dataset) - test_size, test_size])

batch_size = 128  #Change based on GPU capacity

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size*2, num_workers=4, pin_memory=True)

In [4]:
# Input and ouput values of the hidden layers (input the values from the cross-validation step)
architecture = (28,60,30)

# Model definition 
model = DeepNeuralNetwork(4, *architecture)

In [5]:
#Hyperparameters
epochs = 15
max_lr = 0.0001
grad_clip = 0.007
weight_decay = 1e-5
opt_func = torch.optim.Adam

#Training
history = []

history += fit(epochs, max_lr, model, train_loader, test_loader, weight_decay, grad_clip, opt_func)

Epoch [0], last_lr: 0.00002, train_loss: 0.7066, val_loss: 0.7016, val_acc: 0.4346
Epoch [1], last_lr: 0.00004, train_loss: 0.6915, val_loss: 0.6771, val_acc: 0.7761
Epoch [2], last_lr: 0.00008, train_loss: 0.6485, val_loss: 0.6118, val_acc: 0.7761
Epoch [3], last_lr: 0.00010, train_loss: 0.5568, val_loss: 0.5137, val_acc: 0.7761
Epoch [4], last_lr: 0.00010, train_loss: 0.4828, val_loss: 0.4778, val_acc: 0.7761
Epoch [5], last_lr: 0.00010, train_loss: 0.4582, val_loss: 0.4637, val_acc: 0.7761
Epoch [6], last_lr: 0.00009, train_loss: 0.4471, val_loss: 0.4559, val_acc: 0.7761
Epoch [7], last_lr: 0.00008, train_loss: 0.4404, val_loss: 0.4513, val_acc: 0.7761
Epoch [8], last_lr: 0.00006, train_loss: 0.4374, val_loss: 0.4480, val_acc: 0.7803
Epoch [9], last_lr: 0.00005, train_loss: 0.4344, val_loss: 0.4459, val_acc: 0.7858
Epoch [10], last_lr: 0.00003, train_loss: 0.4327, val_loss: 0.4446, val_acc: 0.7853
Epoch [11], last_lr: 0.00002, train_loss: 0.4307, val_loss: 0.4438, val_acc: 0.7850
Ep

In [ ]:
plot_losses(history)

In [ ]:
#This step creates a readable model for a variety of programing languages.
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save('model.pt') # Save
#ONNX export
dummy_input = torch.randn() #cambiar luego
torch.onnx.export(model, dummy_input, 'model.onnx', verbose=True)